In [333]:
import re
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')
from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import itertools
import os

In [334]:
df_news = pd.read_csv(r'C:\Users\Pavlina\Desktop\POC\Combined_News_DJIA.csv')

In [335]:
df_news.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [338]:
from textblob import TextBlob
import re

def clean(headline):
    
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", headline).split())

def sentiment(headline):
   
    analysis = TextBlob(clean(headline))
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1

In [339]:
#Sentiment polarity using TextBlob
from textblob import TextBlob

def sentiment(headline):
   
    analysis = TextBlob(clean(headline))
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1

In [340]:
topn_headlines = 20
top_num = range(1,topn_headlines+1)
dataframe_col = 'sum_sent' + str(max(top_num))
df_news[dataframe_col] = 0
for ii in top_num:
    read_col = 'Top' + str(ii)
    write_col = 'Top' + str(ii) + '_sent'
    df_news[write_col] = np.array([ sentiment(headline) for headline in df_news[read_col] ])
    df_news[dataframe_col] = df_news[dataframe_col] + df_news[write_col]
df_news

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top11_sent,Top12_sent,Top13_sent,Top14_sent,Top15_sent,Top16_sent,Top17_sent,Top18_sent,Top19_sent,Top20_sent
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,1,1,-1,-1,0,1,0,-1,1,0
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,0,1,0,1,0,-1,1,0,0,0
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,1,-1,0,1,1,1,1,-1,-1,1
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,-1,0,1,0,1,-1,0,0,0,0
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,0,1,1,1,-1,1,0,1,0,1
5,2008-08-15,1,"b""Mom of missing gay man: Too bad he's not a 2...","b""Russia: U.S. Poland Missile Deal Won't Go 'U...","b""The government has been accused of creating ...",b'The Italian government has lashed out at an ...,b'Gorbachev: Georgia started conflict in S. Os...,"b""China fakes more than your girlfriend; 'Ethn...","b""The UN's criticism of freedom of expression ...",b'Russian general threatens nuclear strike on ...,...,0,1,1,0,0,1,0,-1,-1,0
6,2008-08-18,0,"b'In an Afghan prison, the majority of female ...","b""Little girl, you're not ugly; they are""","b""Pakistan's Musharraf to Resign, Leave the Co...","b'Tornado throws a bus in Poland, captured by ...","b""Britain's terror laws have left me and my fa...","b""Iran 'fires satellite into space'""",b'Rights of Non-Muslims restricted by new Mald...,b'Tour of Tskhinvali undercuts Russian version...,...,0,0,-1,-1,0,1,1,-1,0,0
7,2008-08-19,0,"b""Man arrested and locked up for five hours af...",b'The US missile defence system is the magic p...,b'Schrder lambasted for blaming Russian confli...,b'Officials: 10 French soldiers killed near Ka...,b'These ten laws make China a totalitarian was...,b'Russia seizes US vehicles',"b""Muslims are only 4% of Denmark's 5.4 million...",b'Taliban Forces Kill 10 French Soldiers and R...,...,0,1,0,0,0,-1,1,1,0,0
8,2008-08-20,1,b'Two elderly Chinese women have been sentence...,b'The Power of Islam: The Human Rights Council...,"b""We had 55 times more military soldiers in th...","b'""I live here on less than a dollar a month"" ...",b'Russia sends aircraft carrier to Syria.',b'The American people should be eternally grat.

In [341]:
headlines_columns = range(1,21)
x_data_list = []
for rows in range(len(df_news)):
    x_dataTemp = []
    for i in range(1,21):
        x_dataTemp.append(df_news.loc[rows,'Top'+str(i)+'_sent'])
    x_data_list.append(x_dataTemp)
x_data = np.array(x_data_list)
y_label = df_news['Label']  

In [342]:
# Y value converted to categotical 
from keras.utils.np_utils import to_categorical
y_label = to_categorical(y_label)


In [343]:
#Array of average daily sentiment score
x_data2 = x_data
x_data2=np.mean(x_data2, axis=1)

In [344]:
# break the dataset to to 65% training and 35% test.
train_sample_perc = .65
n_samples = len(y_label)
x_train = x_data_list[:int(train_sample_perc * n_samples)]
y_train = y_label[:int(train_sample_perc * n_samples)]
x_test = x_data_list[int(train_sample_perc * n_samples):]
y_test = y_label[int(train_sample_perc * n_samples):]

In [345]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix 
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding
import os
import tensorflow as tf

In [346]:
def creat_my_dataset(df):
    x = []
    y = []
    for i in range(50,df.shape[0]):
        x.append(df[i-50:i,0])
        y.append(df[i,0])
    x = np.array(x)
    y = np.array(y)
    return x,y

In [347]:
#LSTM model

In [348]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [349]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))
print(x_train.shape)
print(x_test.shape)

(1292, 20, 1)
(697, 20, 1)


In [350]:
tf.logging.set_verbosity(tf.logging.ERROR)
model = Sequential()
model.add(LSTM(units=96, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=96, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=96))
model.add(Dropout(0.2))
model.add(Dense(units=2, activation='softmax'))


In [351]:
model.compile(optimizer = 'adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [352]:
if (not os.path.exists(r'C:\Users\Pavlina\Desktop\LSTM\LSTM_Model_News_Sentiment_&_Stock_Movement3.h5')):
    model.fit(x_train, y_train, epochs=50, batch_size=32)
    model.save(r'C:\Users\Pavlina\Desktop\LSTM\LSTM_Model_News_Sentiment_&_Stock_Movement3.h5')

Epoch 1/50
1292/1292 [==============================] - 6s 4ms/step - loss: 0.6933 - acc: 0.5348A: 5s - loss: 0.6939 -
Epoch 2/50
1292/1292 [==============================] - 2s 1ms/step - loss: 0.6913 - acc: 0.5379
Epoch 3/50
1292/1292 [==============================] - 2s 1ms/step - loss: 0.6913 - acc: 0.5379
Epoch 4/50
1292/1292 [==============================] - 2s 1ms/step - loss: 0.6918 - acc: 0.5379
Epoch 5/50
1292/1292 [==============================] - 2s 1ms/step - loss: 0.6906 - acc: 0.5379
Epoch 6/50
1292/1292 [==============================] - 2s 1ms/step - loss: 0.6909 - acc: 0.5379
Epoch 7/50
1292/1292 [==============================] - 2s 1ms/step - loss: 0.6917 - acc: 0.5379
Epoch 8/50
1292/1292 [==============================] - 2s 1ms/step - loss: 0.6909 - acc: 0.5410
Epoch 9/50
1292/1292 [==============================] - 2s 1ms/step - loss: 0.6908 - acc: 0.5379
Epoch 10/50
1292/1292 [==============================] - 2s 1ms/step - loss: 0.6905 - acc: 0.5372
Epoch 1

In [353]:
result=model.evaluate(x_test,y_test)
print(result)

697/697 [==============================] - 1s 2ms/step
[0.6950807390117235, 0.5322812044808649]


In [354]:
#SVM model

In [355]:
X = x_data2
y = df_news['Label']

In [ ]:
y.valuescount

In [356]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.35) 

In [357]:
svm1 = SVC(kernel='linear',gamma=2)  
svm1.fit(X_train, y_train)  

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [358]:
y_pred = svm1.predict(X_test) 
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred)) 

[[  0 318]
 [  0 379]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       318
           1       0.54      1.00      0.70       379

   micro avg       0.54      0.54      0.54       697
   macro avg       0.27      0.50      0.35       697
weighted avg       0.30      0.54      0.38       697



C:\Users\Pavlina\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Pavlina\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Pavlina\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [359]:
svm2 = SVC(kernel='poly', degree=8, gamma=2) 
svm2.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=8, gamma=2, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [360]:
y_pred2 = svm1.predict(X_test) 
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))

[[  0 318]
 [  0 379]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       318
           1       0.54      1.00      0.70       379

   micro avg       0.54      0.54      0.54       697
   macro avg       0.27      0.50      0.35       697
weighted avg       0.30      0.54      0.38       697



C:\Users\Pavlina\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Pavlina\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Pavlina\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [361]:
svm3 = SVC(kernel='sigmoid',gamma=2)
svm3.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='sigmoid',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [362]:
y_pred3 = svm1.predict(X_test) 
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))

[[  0 318]
 [  0 379]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       318
           1       0.54      1.00      0.70       379

   micro avg       0.54      0.54      0.54       697
   macro avg       0.27      0.50      0.35       697
weighted avg       0.30      0.54      0.38       697



C:\Users\Pavlina\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Pavlina\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Pavlina\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [363]:
svm4 = SVC(kernel='rbf',gamma=2)
svm4.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [364]:
y_pred4 = svm1.predict(X_test) 
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))

[[  0 318]
 [  0 379]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       318
           1       0.54      1.00      0.70       379

   micro avg       0.54      0.54      0.54       697
   macro avg       0.27      0.50      0.35       697
weighted avg       0.30      0.54      0.38       697



C:\Users\Pavlina\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Pavlina\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Pavlina\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
